# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:

city_weather_df = pd.read_csv("../output_data/Retrieved_City_Weather_Data.csv")
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Okhotsk,59.3833,143.3000,-17.25,76,2,15.28,RU,1640929322
1,Punta Arenas,-53.1500,-70.9167,46.51,76,75,9.22,CL,1640929501
2,Hasaki,35.7333,140.8333,43.09,56,29,20.74,JP,1640929501
3,Hamilton,39.1834,-84.5333,49.98,96,75,4.61,US,1640929215
4,Arraial do Cabo,-22.9661,-42.0278,73.71,91,100,18.30,BR,1640929502
...,...,...,...,...,...,...,...,...,...
552,Lovozero,68.0050,35.0117,15.87,96,100,2.57,RU,1640929701
553,Omsukchan,62.5333,155.8000,-39.39,98,34,3.04,RU,1640929701
554,Caravelas,-17.7125,-39.2481,73.87,91,98,4.61,BR,1640929662
555,Devils Lake,48.1128,-98.8651,-12.05,83,1,1.99,US,1640929702


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

In [5]:
#Use the Lat and Lng as locations and Humidity as the weight
locations_df = city_weather_df[["Lat", "Lng"]].astype(float)
locations_df



,Lat,Lng
0,59.3833,143.3000
1,-53.1500,-70.9167
2,35.7333,140.8333
3,39.1834,-84.5333
4,-22.9661,-42.0278
...,...,...
552,68.0050,35.0117
553,62.5333,155.8000
554,-17.7125,-39.2481
555,48.1128,-98.8651


In [6]:
#Add Heatmap layer to map
humidity_df = city_weather_df["Humidity"].astype(float)
humidity_df


0      76.0
1      76.0
2      56.0
3      96.0
4      91.0
       ... 
552    96.0
553    98.0
554    91.0
555    83.0
556    71.0
Name: Humidity, Length: 557, dtype: float64

In [7]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity_df, 
                                 dissipating=False,max_intensity=100,point_radius = 3)
    
   

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
#Ideal_weather_df=city_weather_df.dropna()
Ideal_weather_df=city_weather_df.copy()

#A max temperature lower than 80 degrees but higher than 70.
Ideal_weather_df = Ideal_weather_df.loc[(Ideal_weather_df["Max Temp"] < 80) & (Ideal_weather_df["Max Temp"] > 70)]


#Wind speed less than 10 mph.
Ideal_weather_df = Ideal_weather_df.loc[(Ideal_weather_df["Wind Speed"] < 10)]
Ideal_weather_df
                                       
# Zero cloudiness.
Ideal_weather_df = Ideal_weather_df.loc[(Ideal_weather_df["Clouds"]==0)]
Ideal_weather_df

  # Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
Ideal_weather_df=Ideal_weather_df.dropna()
Ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
27,Warangal,18.0000,79.5833,78.71,51,0,7.02,IN,1640929429
88,Los Llanos de Aridane,28.6585,-17.9182,71.35,30,0,4.00,ES,1640929530
218,Mundo Nuevo (Ojo de Agua),18.7000,-95.5000,74.10,86,0,6.58,MX,1640929574
249,Tura,25.5198,90.2201,74.43,34,0,4.72,IN,1640929586
402,Maun,-19.9833,23.4167,73.36,83,0,6.91,BW,1640929564


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Store into variable named hotel_df.
hotel_df=Ideal_weather_df

#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
27,Warangal,18.0000,79.5833,78.71,51,0,7.02,IN,1640929429,
88,Los Llanos de Aridane,28.6585,-17.9182,71.35,30,0,4.00,ES,1640929530,
218,Mundo Nuevo (Ojo de Agua),18.7000,-95.5000,74.10,86,0,6.58,MX,1640929574,
249,Tura,25.5198,90.2201,74.43,34,0,4.72,IN,1640929586,
402,Maun,-19.9833,23.4167,73.36,83,0,6.91,BW,1640929564,


In [10]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [11]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    #get hotel based on location, lat and long
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    #extract resukts
    results = response['results']
    
    #Save those name into a dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Cannot find the result")
        print("-------------------------")
       
    


Retrieving Results for Index 27: Warangal.
Closest hotel in Warangal is Hotel Grand Gayathri.
Retrieving Results for Index 88: Los Llanos de Aridane.
Closest hotel in Los Llanos de Aridane is Hotel Hacienda de Abajo.
Retrieving Results for Index 218: Mundo Nuevo (Ojo de Agua).
Cannot find the result
-------------------------
Retrieving Results for Index 249: Tura.
Closest hotel in Tura is Hotel C&C.
Retrieving Results for Index 402: Maun.
Closest hotel in Maun is Cresta Riley's Hotel.


In [12]:
#print the data frame
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
27,Warangal,18.0000,79.5833,78.71,51,0,7.02,IN,1640929429,Hotel Grand Gayathri
88,Los Llanos de Aridane,28.6585,-17.9182,71.35,30,0,4.00,ES,1640929530,Hotel Hacienda de Abajo
218,Mundo Nuevo (Ojo de Agua),18.7000,-95.5000,74.10,86,0,6.58,MX,1640929574,
249,Tura,25.5198,90.2201,74.43,34,0,4.72,IN,1640929586,Hotel C&C
402,Maun,-19.9833,23.4167,73.36,83,0,6.91,BW,1640929564,Cresta Riley's Hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))